In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.pipeline import Pipeline

In [2]:
train=pd.read_csv("train.csv",encoding="latin-1")
test=pd.read_csv("test.csv",encoding="latin-1")

In [3]:
train.head(100)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
95,96,0,mi momacita won't let me go to my bf's bball...
96,97,0,Mom says I have to get a new phone IMMEDIATE...
97,98,0,My new car was stolen....by my mother who wa...
98,99,0,"no hang out with the girls 2day. 2moro, hope..."


In [4]:
X=train["SentimentText"]
Y=train["Sentiment"]
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=.3,stratify=Y)

In [5]:
tfidf=TfidfVectorizer()

In [6]:
stop=stopwords.words("english")

In [7]:
def tokenizer(text):
    return text.split()
porter=PorterStemmer
def tokenizer_porter(text):
    return[porter.stem(word) for word in text.split()]
    
    

In [8]:
def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

print(preprocessor('This!! twit man :) is <b>nice</b>'))
    
    

this twit man is nice :)


In [9]:
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

In [10]:
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [11]:
gs_lr_tfidf.fit(xtrain, ytrain)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


C:\Users\marihan\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
360 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\marihan\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\marihan\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\marihan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, sel

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__preprocessor': [None,
                                                 <function preprocessor at 0x000001FD959AE670>],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'our...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
   

In [12]:
print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__preprocessor': <function preprocessor at 0x000001FD959AE670>, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001FD959AE1F0>}
Best accuracy: 0.772


In [13]:
clf = gs_lr_tfidf.best_estimator_
print('Accuracy in test: %.3f' % clf.score(xtest, ytest))

Accuracy in test: 0.772


In [14]:
twits = [
    "This is really bad, I don't like it at all",
    "I love this!",
    ":)",
    "I'm sad... :("
]

preds = clf.predict(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> {preds[i]}')

This is really bad, I don't like it at all --> 0
I love this! --> 1
:) --> 1
I'm sad... :( --> 0


In [15]:
clf.predict(["i am sad"])[0]

0

In [32]:
from tkinter import*
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
import cv2
from PIL import ImageTk, Image
import tkinter


# create the root window
root = tk.Tk()
root["bg"] = "steelblue"

root.title('Tkinter Open File Dialog')
root.resizable(False, False)
root.geometry('600x600')


    

def prediction():
    predected_lbl=tk.Label(root,text=" ",font=('arial bold',20),fg="white",bg="steelblue")
    pred = clf.predict([str(TextVar.get())])[0]

    lab1=Label(image=photo1,bg="steelblue")
    lab2=Label(image=photo2,bg="steelblue")
  
    if pred==1:
        predected_lbl['text']="positive"
        predected_lbl['fg']="yellow green"
        lab1.pack()
        
        
    if pred==0:
        predected_lbl['text']="nigative"  
        predected_lbl['fg']="red"
        lab2.pack()
        
    predected_lbl.pack()

btn_style=ttk.Style()
btn_style.configure('TButton',font=('tahoma',25),foreground = 'steelblue')

# open button
open_button = ttk.Button(
    root,
    text='Start',
    command=prediction,style="TButton"
)
TextVar=tkinter.StringVar()
Text= tk.Label(root,text='enter feedback',font=('arial bold',30),fg="white",bg="steelblue")
Text_entry=ttk.Entry(root,font=('arial',20),width="40",textvariable=str(TextVar))


img  = Image.open("positive1-removebg-preview.png")
img = img.resize((100,100), Image.ANTIALIAS)
photo1=ImageTk.PhotoImage(img)

img2  = Image.open("negative1-removebg-preview.png")
img2 = img2.resize((100,100), Image.ANTIALIAS)
photo2=ImageTk.PhotoImage(img2)

'''
lab=Label(image=photo).pack()
'''
#Text.grid(row=0,column=0)
Text.pack(pady=15)
Text_entry.pack(pady=15)
open_button.pack(pady=15)

# run the application
root.mainloop()


In [17]:
Text.pack??